In [1]:
# Init environment before running a demo notebook.
from resources.utils import *
init_demo()
from resources.utils import * # reload the global vars again

Overriding of current TracerProvider is not allowed
Attempting to instrument while already instrumented


Enter your API key: ········


Auxip service: http://rs-server-adgs:8000
CADIP service: http://rs-server-cadip:8000
Catalog service: http://rs-server-catalog:8000


In [4]:
# Create a test collection and stage a test item
collection = create_test_collection()
item = stage_test_item()

Staging status for 'DCS_01_S1A_20200105072204051312_ch1_DSDB_00000.raw': IN_PROGRESS
Staging status for 'DCS_01_S1A_20200105072204051312_ch1_DSDB_00000.raw': DONE




In [3]:
# TEMP !!
s3_client = get_s3_client()
for bucket in RSPY_TEMP_BUCKET, RSPY_CATALOG_BUCKET:
    try:
        s3_client.create_bucket(Bucket=bucket)
    except (s3_client.exceptions.BucketAlreadyExists, s3_client.exceptions.BucketAlreadyOwnedByYou):
        pass # do nothing if already exists

In [31]:
import requests

# In this cell, we will call all the catalog GET endpoints and check that 
# the stac authentication extension is set in each JSON output value.

# We extract the GET endpoint paths from the openapi.json specification
openapi_endpoint = "/api" # if local_mode else "/docs/openapi.json" # TEMP !!
openapi = requests.get(f"{stac_client.href_catalog}{openapi_endpoint}", **apikey_headers).json()
get_endpoints = [
    key
    for key, value in openapi["paths"].items()
    if key.startswith("/catalog") and ("get" in value)
]

# Call the endpoints with and without the owner_id: prefix
if cluster_mode:
    implicit_ownerid = []
    for endpoint in get_endpoints:
         implicit_ownerid.append(endpoint)
         if "{owner_id}:{collection_id}" in endpoint:
             implicit_ownerid.append(endpoint.replace("{owner_id}:{collection_id}", "{collection_id}"))
    get_endpoints = implicit_ownerid

# Also test the POST search requests 
post_endpoints = [
    "/catalog/search",
    "/catalog/collections/{owner_id}:{collection_id}/search",
    "/catalog/collections/{collection_id}/search",
]

# Now call each endpoint with the right request function
for method, endpoint in \
    [["GET", endpoint] for endpoint in get_endpoints] + \
    [["POST", endpoint] for endpoint in post_endpoints] \
:
    # Don't call these ones, they don't implement the authentication extension
    if endpoint in [
        "/catalog/api", # openapi.json specification
        "/catalog/api.html", # swagger page
        "/catalog/docs/oauth2-redirect", # don't really know what this is
        "/catalog/conformance", # returns the conformsTo links
        # don't really know what the queryables do, but the authentication extension 
        # is not implemented for these.
        "/catalog/queryables",
        "/catalog/collections/{owner_id}:{collection_id}/queryables",
        "/catalog/collections/{collection_id}/queryables",
    ]:
        print(f"{method} {endpoint}\n{'='*(len(method+endpoint)+1)}\n(ignored)\n")
        continue

    # Replace the {owner_id}, {collection_id}, {item_id} by the right values
    endpoint = endpoint.format(
        owner_id=stac_client.owner_id, collection_id=collection.id, item_id=item.id,
    )
    print(f"{method} {endpoint}\n{'='*(len(method+endpoint)+1)}")

    # Call the endpoint, print STAC output
    if method == "GET":
        stac = requests.get(f"{stac_client.href_catalog}{endpoint}", **apikey_headers).json()
    else:
        stac = requests.post(f"{stac_client.href_catalog}{endpoint}", json={}, **apikey_headers).json()
    pretty_print(stac)

    # Check extension for the root element + nested collections and items
    elements = [stac] + stac.get("collections", []) + stac.get("features", [])
    for stac in elements:
    
        # In cluster mode, we check that the extension is implemented
        if cluster_mode:
            assert "https://stac-extensions.github.io/authentication/v1.1.0/schema.json" in stac["stac_extensions"]
            assert "auth:schemes" in stac
            for link in stac.get("links", []):
                assert link["auth:refs"] == ["apikey"]
            for asset in stac.get("assets", {}).values():
                assert asset["auth:refs"] == ["apikey"]
    
        # In local mode, we check that it is NOT implemented
        else:
            assert "https://stac-extensions.github.io/authentication/v1.1.0/schema.json" not in stac.get("stac_extensions", [])
            assert "auth:schemes" not in stac
    
            for link in stac.get("links", []):
                assert "auth:refs" not in link
    
            for asset in stac.get("assets", {}).values():
                assert "auth:refs" not in asset

        print(f"{stac.get('id', 'Root element')} is OK") 
    print()

GET /catalog/api
(ignored)

GET /catalog/api.html
(ignored)

GET /catalog/docs/oauth2-redirect
(ignored)

GET /catalog/
{
  "type": "Catalog",
  "id": "stac-fastapi",
  "title": "stac-fastapi",
  "description": "stac-fastapi",
  "stac_version": "1.0.0",
  "conformsTo": [
    "https://api.stacspec.org/v1.0.0/item-search",
    "https://api.stacspec.org/v1.0.0-rc.2/item-search#context",
    "http://www.opengis.net/spec/cql2/1.0/conf/cql2-text",
    "https://api.stacspec.org/v1.0.0/collections",
    "http://www.opengis.net/spec/ogcapi-features-1/1.0/conf/oas30",
    "https://api.stacspec.org/v1.0.0/ogcapi-features",
    "https://api.stacspec.org/v1.0.0/item-search#query",
    "https://api.stacspec.org/v1.0.0/core",
    "http://www.opengis.net/spec/ogcapi-features-3/1.0/conf/filter",
    "https://api.stacspec.org/v1.0.0/item-search#fields",
    "http://www.opengis.net/spec/ogcapi-features-3/1.0/conf/features-filter",
    "http://www.opengis.net/spec/cql2/1.0/conf/basic-cql2",
    "https://a

KeyError: 'stac_extensions'

Failed to export metrics to tempo:4317, error code: StatusCode.UNIMPLEMENTED
Failed to export metrics to tempo:4317, error code: StatusCode.UNIMPLEMENTED
Failed to export metrics to tempo:4317, error code: StatusCode.UNIMPLEMENTED
Failed to export metrics to tempo:4317, error code: StatusCode.UNIMPLEMENTED
